In [3]:
import geopandas as gpd
from shapely.geometry import CAP_STYLE, JOIN_STYLE

# === USER PARAMETERS ===
buffer_dist = 10  # Example: 5 meters
frac_buf_dist = 0.05
old_path = "/home/adelb/Downloads/bati-2015.geojson"
new_path = "/home/adelb/Downloads/bati-2022.geojson"


# === Load and clean GeoJSONs ===
gdf_old = gpd.read_file(old_path).buffer(0)
gdf_new = gpd.read_file(new_path).buffer(0)

gdf_old = gpd.GeoDataFrame(geometry=gdf_old, crs="EPSG:3857")
gdf_new = gpd.GeoDataFrame(geometry=gdf_new, crs="EPSG:3857")

# === Apply positive buffer with FLAT ends and MITRE corners ===
gdf_old_buf = gdf_old.geometry.apply(lambda geom: geom.buffer(
    buffer_dist,
    cap_style=CAP_STYLE.flat,
    join_style=JOIN_STYLE.mitre
))
gdf_new_buf = gdf_new.geometry.apply(lambda geom: geom.buffer(
    buffer_dist,
    cap_style=CAP_STYLE.flat,
    join_style=JOIN_STYLE.mitre
))

gdf_old_buf = gpd.GeoDataFrame(geometry=gdf_old_buf, crs=gdf_old.crs)
gdf_new_buf = gpd.GeoDataFrame(geometry=gdf_new_buf, crs=gdf_new.crs)

# === Compute buffered differences ===
buf_demolitions = gpd.overlay(gdf_old_buf, gdf_new_buf, how="difference")
buf_constructions = gpd.overlay(gdf_new_buf, gdf_old_buf, how="difference")

# === Apply negative buffer to remove added margin ===
neg_buf = -(1 + frac_buf_dist) * buffer_dist

demolitions = buf_demolitions.geometry.apply(lambda geom: geom.buffer(
    neg_buf,
    cap_style=CAP_STYLE.flat,
    join_style=JOIN_STYLE.mitre
))
constructions = buf_constructions.geometry.apply(lambda geom: geom.buffer(
    neg_buf,
    cap_style=CAP_STYLE.flat,
    join_style=JOIN_STYLE.mitre
))

demolitions = gpd.GeoDataFrame(geometry=demolitions, crs=gdf_old.crs)
constructions = gpd.GeoDataFrame(geometry=constructions, crs=gdf_old.crs)

# === Optional: Save to files ===
demolitions.to_file("noisy_demolitions.geojson", driver="GeoJSON")
constructions.to_file("noisy_constructions.geojson", driver="GeoJSON")


/tmp/ipykernel_27437/3132260122.py:34: UserWarning: `keep_geom_type=True` in overlay resulted in 14 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  buf_demolitions = gpd.overlay(gdf_old_buf, gdf_new_buf, how="difference")
/tmp/ipykernel_27437/3132260122.py:35: UserWarning: `keep_geom_type=True` in overlay resulted in 25 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  buf_constructions = gpd.overlay(gdf_new_buf, gdf_old_buf, how="difference")


In [ ]:
import geopandas as gpd
from shapely.geometry import box, Polygon, MultiPolygon
from shapely.ops import unary_union
from sklearn.cluster import DBSCAN
import numpy as np


def filter_comparison_by_intersection(reference_path, comparison_path, inter_frac=0.5):
    # Read GeoJSON files
    reference_gdf = gpd.read_file(reference_path)
    comparison_gdf = gpd.read_file(comparison_path)

    # Fix invalid geometries
    reference_gdf['geometry'] = reference_gdf.geometry.buffer(0)
    comparison_gdf['geometry'] = comparison_gdf.geometry.buffer(0)

    # Ensure CRS match
    if reference_gdf.crs != comparison_gdf.crs:
        comparison_gdf = comparison_gdf.to_crs(reference_gdf.crs)

    # Store geometries to keep
    kept_geoms = []

    for comp_geom in comparison_gdf.geometry:
        intersected = reference_gdf[reference_gdf.intersects(comp_geom)]

        keep = False
        for ref_geom in intersected.geometry:
            if ref_geom.area == 0:
                continue  # Avoid division by zero
            intersection_area = comp_geom.intersection(ref_geom).area
            if (intersection_area / ref_geom.area) > inter_frac:
                keep = True
                break

        if keep:
            kept_geoms.append(comp_geom)

    # Create new GeoDataFrame with kept geometries
    kept_gdf = gpd.GeoDataFrame(geometry=kept_geoms, crs=comparison_gdf.crs)
    return kept_gdf


def cluster_and_bbox_flexible_merged(
    geojson_path,
    area_thresh=10.0,
    max_distance=20.0,
    num_bati=3,
    minimal_bbox_area=100.0
):
    # Step 1: Read and fix invalid geometries
    gdf = gpd.read_file(geojson_path)
    gdf['geometry'] = gdf.geometry.buffer(0)

    # Step 2: Filter small polygons
    gdf = gdf[gdf.geometry.area >= area_thresh].reset_index(drop=True)
    if gdf.empty:
        return gpd.GeoDataFrame(columns=['geometry'], geometry='geometry', crs=gdf.crs)

    gdf['cluster'] = -1
    cluster_id = 0
    remaining = gdf.copy()

    # Step 3: Cluster with decreasing min_samples
    for min_samples in range(num_bati, 0, -1):
        if remaining.empty:
            break

        centroids = np.array([[geom.centroid.x, geom.centroid.y] for geom in remaining.geometry])
        clustering = DBSCAN(eps=max_distance, min_samples=min_samples).fit(centroids)

        labels = clustering.labels_
        remaining['temp_cluster'] = labels

        for label in set(labels):
            if label == -1:
                continue
            mask = remaining['temp_cluster'] == label
            if mask.sum() >= min_samples:
                gdf.loc[remaining[mask].index, 'cluster'] = cluster_id
                cluster_id += 1

        remaining = remaining[remaining['temp_cluster'] == -1].drop(columns='temp_cluster')

    # Step 4: Get bboxes for valid clusters
    clustered = gdf[gdf['cluster'] != -1]
    cluster_bboxes = [box(*unary_union(group.geometry).bounds) for _, group in clustered.groupby('cluster')]

    # Step 5: Add bboxes for large unclustered geometries
    unclustered = gdf[gdf['cluster'] == -1]
    large_noise_bboxes = [
        box(*geom.bounds)
        for geom in unclustered.geometry
        if geom.area >= 2 * area_thresh
    ]

    # Step 6: Merge overlapping bounding boxes
    all_bboxes = cluster_bboxes + large_noise_bboxes
    merged_union = unary_union(all_bboxes)

    if isinstance(merged_union, Polygon):
        merged_bboxes = [merged_union]
    elif isinstance(merged_union, MultiPolygon):
        merged_bboxes = list(merged_union.geoms)
    else:
        merged_bboxes = []

    # Step 7: Filter out small bboxes
    filtered_bboxes = [
        poly for poly in merged_bboxes if poly.area >= minimal_bbox_area
    ]

    # Step 8: Return as GeoDataFrame
    bbox_gdf = gpd.GeoDataFrame(geometry=filtered_bboxes, crs=gdf.crs)
    return bbox_gdf


In [5]:
filtered_gdf = filter_comparison_by_intersection(new_path, 'noisy_constructions.geojson', inter_frac=0.4)
filtered_gdf.to_file('cleaned_constructions.geojson', driver='GeoJSON')


In [ ]:
bbox_gdf = cluster_and_bbox_flexible_merged(
    "cleaned_constructions.geojson",
    area_thresh=60,
    max_distance=100.0,
    num_bati=7,
    minimal_bbox_area= 300,
)
bbox_gdf.to_file("bbox_constructions_flexible_merged.geojson", driver="GeoJSON")
